# Comparative Impact Analysis of Proposed Endoscopy Sites by Cluster Count (k = 3 to 7)

This notebook evaluates the spatial and service impact of proposed endoscopy sites generated via clustering underserved LSOAs with cluster counts ranging from 3 to 7. It compares how cluster resolution affects:

- Geographic coverage and distribution of proposed sites
- Accessibility improvements measured by reachable endoscopy rooms within 60 minutes
- Competition-adjusted service capacity per 100k population aged 50–74
- Accessibility tier changes (Low, Marginal, Adequate, No Access)

The analysis informs the choice of cluster number for strategic planning of new facilities balancing coverage, efficiency, and equity.

---
